In [1]:

# module import
import os
import sys
from datetime import datetime
import pickle

import numpy as np
import PIL.Image
import torch
import torchvision
                
sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel, save_video, normalise_vid, get_cnn_features, img_deprocess, get_target_feature_shape
from activation_maximization import generate_preferred

In [2]:
# network load
net = torchvision.models.alexnet(pretrained=True)

In [3]:
# image mean and std for pre/de-process image for input network
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

# if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
norm = 255

In [4]:
#save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_simpleCNN_conv'
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [5]:
# initial image for the optimization
h, w = 224,224
initial_input = np.random.randint(0, 256, (h,w,3))

In [6]:
net

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [7]:
target_layer = "net.features[8]"
target_layer = "net.features[10]"

In [8]:
#target layer setting
exec_str_list = [target_layer +".register_forward_hook(hook)"]
## obtain target feature shape
# transform input shape for torch avairable shape
initial_torch_input = torch.Tensor(initial_input.transpose(2,0, 1)[np.newaxis])
# obtain target layer activation 
feat_shape = get_target_feature_shape(net, initial_torch_input, exec_str_list)

In [9]:
# options
opts = {
    'img_mean': img_mean, #img_mean to preprocessing input image
    'img_std': img_std,   #img_std to preprocessing input image
    'norm': norm,         #if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
    
    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start': 1., # learning rate
    'lr_end': 1.,   # we can change learning rate linearly setteing these two parameters

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,   # we can change momentum linearly setteing these two parameters too

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,   # we can also change pixel decay linealy 

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during optimization
    'jitter_size': 4,
    
    'use_p_norm_reg': False, # use p_norm regularization
    'p': 2,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    'input_size': (224,224,3),
    'initial_input': initial_input, # the initial image for the optimization (setting to None will use random noise as initial image)
    }



In [10]:
channel_list = [14,56]

In [11]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # target units
    feat_size = feat_shape
    y_index = int(feat_size[2]/2) # the unit in the center of feature map
    x_index = int(feat_size[3]/2) # the unit in the center of feature map
    feature_mask = np.zeros(feat_size)
    feature_mask[0,channel,y_index,x_index] = 1
    feature_mask[0,channel] = 1
    # weights for the target units
    feature_weight = np.zeros(feat_size, dtype=np.float32)
    feature_weight[:] = 1.
    
    #
    preferred_stim = generate_preferred(net, exec_str_list, feature_mask=feature_mask, **opts)
    # save the results
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.npy'
    np.save(os.path.join(save_path,save_name), preferred_stim)

    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.jpg'
    #save_video(normalise_vid(clip_extreme_pixel(preferred_stim,pct=0.04)), save_name, save_path )
    PIL.Image.fromarray(normalise_img(clip_extreme_pixel(preferred_stim, pct=0.04))).save(
                    os.path.join(save_path, save_name))


channel=14

iter=1; mean(abs(feat))=3.00956;
iter=2; mean(abs(feat))=4.15028;
iter=3; mean(abs(feat))=4.73639;
iter=4; mean(abs(feat))=2.91666;
iter=5; mean(abs(feat))=2.78229;
iter=6; mean(abs(feat))=0.889395;
iter=7; mean(abs(feat))=1.47165;
iter=8; mean(abs(feat))=1.11988;
iter=9; mean(abs(feat))=1.91523;
iter=10; mean(abs(feat))=1.54898;
iter=11; mean(abs(feat))=2.58143;
iter=12; mean(abs(feat))=2.21104;
iter=13; mean(abs(feat))=0.0168529;
iter=14; mean(abs(feat))=2.71372;
iter=15; mean(abs(feat))=2.52517;
iter=16; mean(abs(feat))=0.597162;
iter=17; mean(abs(feat))=2.32196;
iter=18; mean(abs(feat))=1.19449;
iter=19; mean(abs(feat))=1.71786;
iter=20; mean(abs(feat))=0.164479;
iter=21; mean(abs(feat))=0.198081;
iter=22; mean(abs(feat))=2.0235;
iter=23; mean(abs(feat))=1.08697;
iter=24; mean(abs(feat))=2.41633;
iter=25; mean(abs(feat))=1.54338;
iter=26; mean(abs(feat))=1.43098;
iter=27; mean(abs(feat))=5.19131;
iter=28; mean(abs(feat))=4.91938;
iter=29; mean(abs(feat))=5.0558;
iter=3

iter=39; mean(abs(feat))=9.23626;
iter=40; mean(abs(feat))=7.31584;
iter=41; mean(abs(feat))=8.15235;
iter=42; mean(abs(feat))=10.3648;
iter=43; mean(abs(feat))=10.6277;
iter=44; mean(abs(feat))=13.0555;
iter=45; mean(abs(feat))=6.57443;
iter=46; mean(abs(feat))=8.77593;
iter=47; mean(abs(feat))=9.57118;
iter=48; mean(abs(feat))=8.52526;
iter=49; mean(abs(feat))=10.9073;
iter=50; mean(abs(feat))=13.2477;
iter=51; mean(abs(feat))=12.1278;
iter=52; mean(abs(feat))=7.85123;
iter=53; mean(abs(feat))=9.87644;
iter=54; mean(abs(feat))=14.2923;
iter=55; mean(abs(feat))=12.09;
iter=56; mean(abs(feat))=7.36784;
iter=57; mean(abs(feat))=10.5997;
iter=58; mean(abs(feat))=14.9073;
iter=59; mean(abs(feat))=10.9935;
iter=60; mean(abs(feat))=15.1378;
iter=61; mean(abs(feat))=15.9769;
iter=62; mean(abs(feat))=12.0431;
iter=63; mean(abs(feat))=8.50422;
iter=64; mean(abs(feat))=7.85611;
iter=65; mean(abs(feat))=11.375;
iter=66; mean(abs(feat))=10.4672;
iter=67; mean(abs(feat))=9.04333;
iter=68; mean(abs

In [32]:
save_name_npy = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.npy'
np.save(os.path.join(save_path,save_name), preferred_stim)

In [33]:
feature_weight

array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        ...,

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
    